# Generate Oportunities from our data

Here, we will use our database and predict which apartments are the best oportunities, taking into account our prediction from the model and calculating the residuals.

In [53]:
import pandas as pd
import joblib
import sys

sys.path.append('../backend')
from preprocessing import clean_airbnb_data, prepare_for_modeling

print("1. Loading raw data and model...")
raw_data_path = '../data/listings.csv'
model_path = '../backend/models/airbnb_pricing_model.joblib'

1. Loading raw data and model...


In [54]:
df_raw = pd.read_csv(raw_data_path)
model = joblib.load(model_path)

print("2. Applying data cleaning...")
# Clean raw data
df_clean = clean_airbnb_data(df_raw)

2. Applying data cleaning...


In [55]:
df_readable = df_clean.copy()

In [56]:
print("3. Preprocessing features for the model...")
# Preprocessing
df_processed = prepare_for_modeling(df_clean)

3. Preprocessing features for the model...


In [57]:
# Separate X (features) and y (actual price). 
if 'price' in df_processed.columns:
    X = df_processed.drop(columns=['price'])
    y_actual = df_processed['price']
else:
    # If preprocessing drops 'price', we take it from our readable backup
    X = df_processed
    y_actual = df_readable['price']

In [58]:
print("4. Calculating AI valuations...")
predictions = model.predict(X)

4. Calculating AI valuations...


In [59]:
print("5. Analyzing residuals (Value Investing)...")
df_readable['predicted_price'] = predictions

5. Analyzing residuals (Value Investing)...


In [60]:
# Ensure the actual price is in df_readable to calculate the difference
if 'price' not in df_readable.columns:
    df_readable['price'] = y_actual

df_readable['residual'] = df_readable['predicted_price'] - df_readable['price']

In [61]:
print("6. Filtering profitable opportunities...")
# Keep only the bargains (where the predicted price > actual price)
df_bargains = df_readable[df_readable['residual'] > 0].copy()

6. Filtering profitable opportunities...


In [62]:
# Calculate the discount percentage
if not df_bargains.empty:
    df_bargains['discount_pct'] = (df_bargains['residual'] / df_bargains['predicted_price']) * 100

In [63]:
print("7. Generating lightweight CSV for the Web...")
# Select only the columns Streamlit needs to render the UI
required_columns = [
    'listing_url',
    'latitude', 'longitude', 
    'neighbourhood_group_cleansed', # District
    'neighbourhood_cleansed',       # Neighborhood
    'room_type', 'accommodates',    
    'price', 'predicted_price', 'residual', 'discount_pct'
]

# Just in case your dataset has slightly different names, keep only the existing ones
final_columns = [col for col in required_columns if col in df_bargains.columns]

df_export = df_bargains[final_columns].sort_values(by='residual', ascending=False)

output_path = '../data/chollos_madrid.csv'
df_export.to_csv(output_path, index=False)

print(f"✅ Success! Found {len(df_export)} bargains.")
print(f"File with {len(final_columns)} columns saved at: {output_path}")

7. Generating lightweight CSV for the Web...
✅ Success! Found 9234 bargains.
File with 11 columns saved at: ../data/chollos_madrid.csv
